In [1]:
import math
import tkinter as tk
from tkinter import ttk

In [2]:
import pandas as pd

In [3]:
def create_table(canvas):
    global tree

    tree = ttk.Treeview(
        canvas, 
        show="headings",
        selectmode="browse",
        height=10,
        padding=10,
    )
    
    tree["columns"] = ("Enunciado", "Tópico", "Dificuldade", "Resolvido?")
    tree.heading("Enunciado", text="Enunciado")
    tree.heading("Tópico", text="Tópico")
    tree.heading("Dificuldade", text="Dificuldade", command=lambda c="Dificuldade": sort_treeview(tree, c, False))
    tree.heading("Resolvido?", text="Resolvido?")
    tree.bind('<ButtonRelease-1>', on_line_click)

    fill_list(tree)

    tree.place(x=250, y=175)

    print("Created Treeview")
    return tree


In [4]:
def fill_list(tree):
    for index, row in df.iterrows():
        tree.insert("", "end", values=(row["question"], row["topic"], row["difficulty"], row["resolvido"]))

In [5]:
def create_text_box(canvas):
    text_box = tk.Text(
        canvas,
        height=10,  # Set the desired height (adjust as needed)
        width=40,   # Set the desired width (adjust as needed)
    )

    canvas.create_window(550, 300, window=text_box)  # Adjust the coordinates as needed

    text_box.pack(expand=False)
    text_box.insert('end', "Question")
    text_box.config(state='disabled')

    print("Created Text Box")
    return text_box

In [6]:
def fill_text_box(text):
    global text_box
    text_box.delete(0, tk.END)
    text_box.insert(0, text)

In [7]:
def sort_treeview(tree, col, descending):
    difficulty_order = {"Difícil": 0, "Médio": 1, "Fácil": 2}

    data = [(difficulty_order.get(tree.set(item, col), float('inf')), item) for item in tree.get_children('')]
    data.sort(reverse=descending)
    for index, (_, item) in enumerate(data):
        tree.move(item, '', index)
    tree.heading(col, command=lambda: sort_treeview(tree, col, not descending))

In [8]:
def resize_canvas(event):
    global color1, color2
    
    canvas.delete("all")
    
    color1, color2 = set_color()
    
    create_gradient_background(0, 0, event.width, event.height, color1, color2)

In [9]:
def resize_table(event):
    print("CALLED RESIZE TABLE")
    tree_width = event.width * 0.9
    tree_height = event.height * 0.5
    
    tree.place(x=(event.width - tree_width) / 2, y=(event.height - tree_height) / 2, width=tree_width, height=tree_height)

In [10]:
def on_line_click(event):
    selected_item = tree.selection()

    if selected_item:
        item = tree.selection()[0]
        question_text = tree.item(item, "values")[0]
        print("Values for clicked item:", question_text)
        #fill_text_box(question_text)
    else:
        print("No exercise selected")
        #fill_text_box("No exercise selected")

In [11]:
def create_gradient_background(x1, y1, x2, y2, color1, color2):
    # Direção do gradiente
    dx = x2 - x1
    dy = y2 - y1
    
    #Cálculo para interpolação das cores
    steps = max(dx, dy)
    step_r = (color2[0] - color1[0]) / steps
    step_g = (color2[1] - color1[1]) / steps
    step_b = (color2[2] - color1[2]) / steps
    
    
    
    for y in range(y1, y2):
        # Calculando cor
        r = int((color1[0] * (y2 - y) + color2[0] * (y - y1)) / (y2 - y1))
        g = int((color1[1] * (y2 - y) + color2[1] * (y - y1)) / (y2 - y1))
        b = int((color1[2] * (y2 - y) + color2[2] * (y - y1)) / (y2 - y1))
        hex_color = f"#{r:02X}{g:02X}{b:02X}"
        
        canvas.create_line(x1, y, x2, y, fill=hex_color)

In [12]:
def set_color():
    c1 = (107, 31, 124)  # Rosa #A1405D
    c2 = (161, 64, 93)  # Roxo #6B1F7C
    return c1, c2

In [13]:
def create_canvas(root, w, h):
    global canvas
    canvas = ctk.Canvas(root, width=w, height=h)
    canvas.pack(fill=tk.BOTH, expand=True)
    create_gradient_background(0, 0, w, h, color1, color2)

In [14]:
def start_interface(root, w, h):
    global canvas, color1, color2
    global text_box
    canvas = tk.Canvas(root, width=w, height=h, bg="#A1405D", highlightthickness=0)
    canvas.pack(fill=tk.BOTH, expand=True)
    canvas.bind("<Configure>", resize_canvas, resize_table)
    
    color1, color2 = set_color()
    
    create_table(canvas)
    text_box = create_text_box(canvas)
    
    root.mainloop()

In [15]:
def awake_interface():
    #import data_module.ipynb
    read_data()
    root = tk.Tk()
    root.geometry("1280x780")
    root.title("Main Tab")
    height = 780
    width = 1280
    start_interface(root, width, height)

In [16]:
def install_dependencies():
    !pip install tkinter
    !pip install tk
    !pip install pandas

In [17]:
def read_data(): #FROM DATA_MODULE
    global df
    exercises_data_path = './data/exercises.csv'
    df = pd.read_csv(exercises_data_path)

In [18]:
def get_dataframe(): #FROM DATA_MODULE
    return df

In [19]:
def decode_variables(df): #FROM DATA MODULE
    global var_names
    var_names = []
    global var_values
    var_values = []
    for row in df['variables']:
        variables = row.split(';')
        var_dict = {}
        
        for variable in variables:
            var_parts = variable.split('=')
            if len(var_parts) == 2:
                var_name, var_value = var_parts
                var_dict[var_name] = float(var_value) #converte strings em floats
        
        var_names.append(var_dict.keys())
        var_values.append(var_dict.values())
        
    return var_names, var_values

In [20]:
def get_exercise_variables(id):
    global var_names, var_values
    exercise_variable_names = []
    exercise_variable_values = []
    
    for var in var_names[id]:
        exercise_variable_names.append(var)
        
    
    for var in var_values[id]:
        exercise_variable_values.append(var)
    
    print(exercise_variable_names, exercise_variable_values)
    return exercise_variable_names, exercise_variable_values

In [21]:
def main():
    awake_interface()

In [22]:
if __name__ == "__main__":
    main()

Created Treeview
Created Text Box


In [23]:
var_names, var_values = decode_variables(df)

In [24]:
def solve_question_1():
    var_names, var_values = get_exercise_variables(0)
    

    saltos = var_values[var_names.index('saltos')]
    tempo = var_values[var_names.index('tempo')]
    raio = var_values[var_names.index('raio')]
    
    velocidade = (saltos*2*3*(raio/100))/tempo
    #Considerando 1 salto a cada volta, são 105 voltas (n) em 30 s (Δt). v=ΔS/Δt => n*(2πR)/Δt => (105*2*3*0,9)/30 => v=18,9m/s
        
        
    return velocidade

In [25]:
resposta = solve_question_1()
print(resposta)

['saltos', 'tempo', 'raio'] [105.0, 30.0, 90.0]
18.9
